# Visualing portfolio performance

In [12]:
import sys

# Add the src directory to the path
# If it bugs out, just alternate between ../../ and ../../src
sys.path.append("../../")

from src.config import config
from sqlalchemy import create_engine
from sqlalchemy.orm import Session

ENGINE  = create_engine(config.SQLALCHEMY_DATABASE_URI, echo=False, pool_size=20, max_overflow=0)

print('Sandbox:', config.API_SANDBOX)

Sandbox: False


In [13]:
from src.service import DataService
from cryptolib.model import PortfolioHistoryModel, BalanceHistoryModel, CurrencyPairConfigModel
import matplotlib.pyplot as plt

def bot_info(bot: CurrencyPairConfigModel, **params):
    strat_config = ','.join([sc.value for sc in bot.strategy_config])
    output = f"Bot: {bot.id} - {bot.currency_pair} - {bot.strategy.value} ({strat_config}) - {bot.interval.value}"
    for key, val in params.items():
        output += f" - {key}: {val}"
    print(output)

### Bokeh
For nice graphing...

In [15]:
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource
from bokeh.models.tools import HoverTool
from bokeh.palettes import Category10, Dark2_5
from bokeh.io import output_notebook

import itertools

output_notebook()

colors = itertools.cycle(Dark2_5)

class BokehGraph:

    WIDTH = 1400
    HEIGHT = 600

    def __init__(self):
        self.figure = figure(width=self.WIDTH , height=self.HEIGHT, x_axis_type="datetime")
        self.figure.add_tools(HoverTool(
            tooltips=[
                ('symbol', '$name'),
                ('date', '@x{%F}'),
                ('value', '@y{0.0000}'),
            ],
            formatters={
                '@x': 'datetime',
            },
            mode='vline'
        ))

    def add_line(self, x, y, legend_label, name):
        source = ColumnDataSource(data=dict(x=x, y=y))
        self.figure.line(x="x", y="y", source=source, line_width=2, legend_label=legend_label, color=next(colors), name=name)

    def show(self):
        # Hide lines on click
        self.figure.legend.click_policy = "hide"
        show(self.figure)

Loading BokehJS ...

### Bot performance

In [10]:
# Number of active bots
with Session(ENGINE) as session:
    bots = DataService().get_active_bots(session)
    print(len(bots))

1


##### Show profitable bots

In [16]:
from src.service import DataService
from cryptolib.model import CurrencyPairConfigModel, OrderModel
from cryptolib.enums import OrderSide

with Session(ENGINE) as session:
    users = DataService().get_users(session)
    for user in users:
        bots = user.currency_pair_configs
        profit = 0
        for bot in bots:
            orders = bot.orders
            if len(orders) == 0:
                continue

            latest_order: OrderModel = orders[-1]            
            if latest_order.side.value == OrderSide.BUY.value:
                profit += latest_order.cost - bot.allocated_balance
            else:
                profit += latest_order.amount - bot.allocated_balance
        
        print(f"User Id: {user.id} | Strategy: {bots[0].strategy.value} | Interval: {bots[0].interval.value} - P/L: {profit}") if profit > 0 else None

User Id: 1 | Strategy: Bollinger Bands | Interval: 1m - P/L: 3.7841064699999833
User Id: 23 | Strategy: Relative Strength Index | Interval: 1m - P/L: 1942.114556569999
User Id: 24 | Strategy: Chaikin Oscillator | Interval: 1m - P/L: 578.105688319999
User Id: 26 | Strategy: Chaikin Oscillator | Interval: 5m - P/L: 405.1770468099994
User Id: 28 | Strategy: Chaikin Oscillator | Interval: 30m - P/L: 981.2958956699995
User Id: 32 | Strategy: Commodity Channel Index | Interval: 1m - P/L: 801.0754232500003
User Id: 33 | Strategy: Commodity Channel Index | Interval: 5m - P/L: 976.9868413900003
User Id: 35 | Strategy: Commodity Channel Index | Interval: 30m - P/L: 827.8116922400004
User Id: 36 | Strategy: Commodity Channel Index | Interval: 1h - P/L: 1019.3423543099989
User Id: 37 | Strategy: Commodity Channel Index | Interval: 2h - P/L: 125.2463946099997
User Id: 44 | Strategy: Chaikin Oscillator | Interval: 15m - P/L: 186.03620545000012
User Id: 45 | Strategy: Chaikin Oscillator | Interval: 3

**For a particular user**

In [18]:
from cryptolib.model import CurrencyPairConfigModel, OrderModel
from cryptolib.enums import OrderSide

# The user to check
# be mindful of how many active bots are running.
# E.g. plotting 400 bots is pretty slow
user_id = 47

with Session(ENGINE) as session:
    user = DataService().get_user(session, user_id)
    bots: list[CurrencyPairConfigModel] = user.currency_pair_configs
    
    print("=====================================")
    print(f"User: {user.email}")
    
    # Plot the history of orders for each bot in bokeh
    graph = BokehGraph()
    fee = 1 # euro
    fee_sum = 0
    for i, bot in enumerate(bots):
        orders: list[OrderModel] = bot.orders
        print('orders:', len(orders))
        x = [o.created_at for o in orders if o.side.value == OrderSide.BUY.value]
        y = [o.cost for o in orders if o.side.value == OrderSide.BUY.value]

        # calculate fees
        fee_sum += len(orders) * fee
               
        divisor = len(y) if len(y) > 0 else 1
        avg = sum(y) / divisor

        # Skip plotting the losing bots
        if avg < bot.allocated_balance:
            # Disable the bots if desired
            # bot.is_active = False
            # session.commit()
            continue
        
        bot_info(bot, avg=avg)
        graph.add_line(x, y, f"Bot {i} - {bot.currency_pair}", name=f"{bot.currency_pair}")

    print('fee_sum:', fee_sum)
    graph.show()


User: bot47@admin.com
orders: 486
Bot: 943 - AXSUSDT - Relative Strength Index (1) - 1m - avg: 5755.527377477369
orders: 556
Bot: 942 - MINAUSDT - Relative Strength Index (1) - 1m - avg: 6041.970220635894
orders: 429
Bot: 944 - TRUUSDT - Relative Strength Index (1) - 1m - avg: 5888.857883935163
fee_sum: 1471


In [46]:
# Calculate binance trading fees

# The user to check
user_id = 47

with Session(ENGINE) as session:
    user = DataService().get_user(session, user_id)
    bots: list[CurrencyPairConfigModel] = user.currency_pair_configs
    
    print("=====================================")
    print(f"User: {user.email}")
    
    fee = 0.00075
    fee_sum = 0
    for i, bot in enumerate(bots):
        orders: list[OrderModel] = bot.orders
        print('orders:', len(orders))
        # get every second orders
        orders = orders[::2]
        print('orders:', len(orders))

        # calculate fees
        fees = sum([o.cost * fee for o in orders if o.side.value == OrderSide.BUY.value])
        fees += sum([o.amount * fee for o in orders if o.side.value == OrderSide.SELL.value])
        fee_sum += fees
        print('fees:', fees)
               
    print('fee_sum:', fee_sum)

User: bot47@admin.com
orders: 473
orders: 237
fees: 1019.4776537920199
orders: 544
orders: 272
fees: 1227.7487063334456
orders: 421
orders: 211
fees: 929.5401659415382
fee_sum: 3176.7665260670037
